In [24]:
import yfinance as yf
import pandas as pd 
import os
import numpy as np
import pickle

In [13]:
netflix = yf.download(tickers='NFLX')
netflix.index

[*********************100%***********************]  1 of 1 completed


DatetimeIndex(['2002-05-23', '2002-05-24', '2002-05-28', '2002-05-29',
               '2002-05-30', '2002-05-31', '2002-06-03', '2002-06-04',
               '2002-06-05', '2002-06-06',
               ...
               '2024-08-23', '2024-08-26', '2024-08-27', '2024-08-28',
               '2024-08-29', '2024-08-30', '2024-09-03', '2024-09-04',
               '2024-09-05', '2024-09-06'],
              dtype='datetime64[ns]', name='Date', length=5611, freq=None)

In [15]:
netflix_df = netflix[netflix.index > '31-12-2022'].copy()
netflix_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-03,298.059998,298.390015,288.700012,294.950012,294.950012,6764000
2023-01-04,298.239990,311.140015,295.510010,309.410004,309.410004,9345100
2023-01-05,307.000000,314.179993,304.549988,309.700012,309.700012,8328400
2023-01-06,311.570007,316.769989,303.690002,315.549988,315.549988,8959800
2023-01-09,316.829987,321.700012,313.220001,315.170013,315.170013,6766600


In [17]:
netflix_df['change_tmr_%'] = netflix_df.Close.pct_change(-1) * 100 * -1
netflix_df['Tmr_Projection'] = np.where(netflix_df['change_tmr_%'] > 0 , "UP", "DOWN" )

In [18]:
os.makedirs('data', exist_ok=True)
netflix_df.to_csv("data/NFLX_Processed_Data.csv", index= True)

In [19]:
target = netflix_df['Tmr_Projection']
predictor = netflix_df.drop(columns=['change_tmr_%','Tmr_Projection'])

In [20]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X=predictor, y=target)

RandomForestClassifier()

In [23]:
y_pred = model.predict(predictor)
netflix_df['predicted'] = y_pred
netflix_df.head()

,Open,High,Low,Close,Adj Close,Volume,change_tmr_%,Tmr_Projection,predicted
Date,,,,,,,,,
2023-01-03,298.059998,298.390015,288.700012,294.950012,294.950012,6764000,4.673408,UP,UP
2023-01-04,298.239990,311.140015,295.510010,309.410004,309.410004,9345100,0.093642,UP,UP
2023-01-05,307.000000,314.179993,304.549988,309.700012,309.700012,8328400,1.853898,UP,UP
2023-01-06,311.570007,316.769989,303.690002,315.549988,315.549988,8959800,-0.120562,DOWN,DOWN
2023-01-09,316.829987,321.700012,313.220001,315.170013,315.170013,6766600,3.776636,UP,UP


In [25]:
os.makedirs('model', exist_ok=True)
model_path = open('model/netflix_model.pkl', 'wb')
pickle.dump(model, model_path)

In [26]:
from backtesting import Backtest, Strategy
class ClassificationUpDown(Strategy):
    def init(self):
        self.model = model
        self.already_bought = False
    
    def next(self):
        explanatory_today = self.data.df.tail(1)
        forecast_tmr = model.predict(explanatory_today)[0]
        if forecast_tmr == 'UP' and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tmr == 'DOWN' and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

In [28]:
bt = Backtest(data = predictor, strategy=ClassificationUpDown, cash=10000, commission=.002, exclusive_orders=True )
result1 = 
result = result1.to_frame(name='Value')
result


,Value
Start,2023-01-03 00:00:00
End,2024-09-06 00:00:00
Duration,612 days 00:00:00
Exposure Time [%],99.526066
Equity Final [$],587858.214254
Equity Peak [$],590556.720457
Return [%],5778.582143
Buy & Hold Return [%],124.834717
Return (Ann.) [%],1039.022879
Volatility (Ann.) [%],334.507503
